In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import bokeh as bk
%matplotlib inline

from pylab import rcParams
rcParams['figure.figsize'] = 12,5
rcParams['xtick.labelsize'] = 12
rcParams['ytick.labelsize'] = 12
rcParams['axes.titlesize'] = 14

import warnings
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)

#view all the columns in the dataframe
pd.options.display.max_columns = None

In [ ]:
#Load train and test data
sales = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv',parse_dates=True)
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

#Loading other files
item_categories = pd.read_csv('../input/predict-future-sales-eng-translation/categories.csv')
items = pd.read_csv('../input/predict-future-sales-eng-translation/items.csv')
shops = pd.read_csv('../input/predict-future-sales-eng-translation/shops.csv')

In [ ]:
sales.head()

In [ ]:
sales.info()

In [ ]:
sales[['item_price','item_cnt_day']].describe()

In [ ]:
#Check for Null values
sales.isnull().sum()

In [ ]:
#Check for Duplicates
len(sales[sales.duplicated()==True])

In [ ]:
#Dropping the duplicates
sales.drop_duplicates(keep='first', inplace=True)

In [ ]:
#View test data
test.head()

In [ ]:
shops.head()

In [ ]:
item_categories.head()

In [ ]:
items.head()

In [ ]:
shops['shop_name'].nunique()

In [ ]:
item_categories['category_id'].nunique()

In [ ]:
items['item_id'].nunique()

In [ ]:
sales['shop_id'].value_counts().sort_values(ascending = False)

In [ ]:
sales['item_id'].value_counts().sort_values(ascending = False)

In [ ]:
#EDA
#Histogram of Item price
sns.distplot(sales['item_price'])
plt.title('Histogram of the Item Price')
plt.show()

In [ ]:
sns.distplot(sales['item_price'])
plt.xscale('log')
plt.title('Histogram of Item Price (log scale)')
plt.show()

In [ ]:
sns.boxplot(sales['item_price'],orient='h')
plt.xscale('log')
plt.title('Box plot of the Item Price (log scale)')
plt.grid()
plt.show()

In [ ]:
print(f"Median item price : {np.median(sales.item_price)}")

In [ ]:
#Interquartile range
q3,q1 = np.percentile(sales['item_price'],[75,25])
print(f"q1 : {q1}")
print(f"q3 : {q3}")
print(f"Interquartile Range (IQR) : {q3 - q1}")

In [ ]:
df1 = sales[(sales['item_price']<1000) & (sales['item_price']>=249)]
sns.distplot(df1['item_price'])
plt.title('Histogram of Item Price')
plt.show()

In [ ]:
sales[sales['item_price']<0]

In [ ]:
test.head()


In [ ]:
sales.info()

In [ ]:
IQR = q3 - q1
upper = np.where(sales['item_price']>= (q3+1.5*IQR))

#lower = np.where(sales['item_price']<= (q1-1.5*IQR))
sales.drop(upper[0], inplace = True)
sales.info()

In [ ]:
test['date_block_num'] = 34
test.head()

In [ ]:
data_concat = pd.concat([sales,test])
data_concat.head()

In [ ]:
data_concat.drop(['ID','date'],axis=1,inplace=True)
data_concat.head()

In [ ]:
data = data_concat.groupby(['date_block_num','shop_id','item_id'],as_index=False)['item_cnt_day'].apply(sum)
data.head()

In [ ]:
data['shop_lag_1'] = data.groupby('shop_id')['item_cnt_day'].shift(1)
data['shop_lag_2'] = data.groupby('shop_id')['item_cnt_day'].shift(2)

data['item_lag_1'] = data.groupby('item_id')['item_cnt_day'].shift(1)
data['item_lag_2'] = data.groupby('item_id')['item_cnt_day'].shift(2)

In [ ]:
data[data['shop_id']==2]['item_id'].value_counts().sort_values(ascending=False)[:20]

In [ ]:
data['shop_median'] = data.groupby('shop_id')['item_cnt_day'].median()
data['shop_mean'] = data.groupby('shop_id')['item_cnt_day'].mean()

data['item_median'] = data.groupby('item_id')['item_cnt_day'].median()
data['item_mean'] = data.groupby('item_id')['item_cnt_day'].mean()

data.head()

In [ ]:
data.describe().transpose()

In [ ]:
data.isna().sum()

In [ ]:
data.fillna(0.0,inplace=True)
data.isna().sum()

In [ ]:
test_data = data[data['date_block_num']==34]
data_new = data[data['date_block_num']!=34]

split_ratio = 0.80
train_data = data_new[:int(split_ratio*len(data_new))]
valid_data = data_new[len(train_data):]

train_data.shape, test_data.shape,valid_data.shape

In [ ]:
X_train = train_data.drop('item_cnt_day',axis=1)
y_train = train_data['item_cnt_day']

X_valid = valid_data.drop('item_cnt_day',axis=1)
y_valid = valid_data['item_cnt_day']

X_test = test_data.drop('item_cnt_day',axis=1)
y_test = test_data['item_cnt_day']

In [ ]:
from sklearn.preprocessing import (StandardScaler,
                                   MinMaxScaler,
                                   PowerTransformer,PolynomialFeatures)
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
#evaluation metrics
from sklearn.metrics import mean_squared_error

In [ ]:
def test_submission(data,model):
    
    predictions = model.predict(data)
    submission = test['ID'].to_frame()
    submission['item_cnt_month'] = predictions
    submission.to_csv('submission.csv', index=False)
    

In [ ]:
preprocess = Pipeline([
    ('scaler',StandardScaler()), ('decompose',PCA(n_components=0.90))
])

#('poly_features',PolynomialFeatures(degree=2)),


X_train = preprocess.fit_transform(X_train)
X_valid = preprocess.transform(X_valid)
X_test = preprocess.transform(X_test)

In [ ]:
lr = LinearRegression()
rf = RandomForestRegressor(max_depth=3, random_state=42)
gb = GradientBoostingRegressor(random_state=42)


In [ ]:
def modeling(model,X_train=X_train,y_train=y_train,X_valid=X_valid,y_valid=y_valid,X_test=X_test,y_test=y_test):
    
    
    model.fit(X_train,y_train)
    train_pred = model.predict(X_train)
    valid_pred = model.predict(X_valid)
    test_pred = model.predict(X_test)
    
    print(f"Model Name : {model}")
    print(f"RMSE on the train data : {np.sqrt(mean_squared_error(y_train,train_pred))}")
    print(f"RMSE on the validation data : {np.sqrt(mean_squared_error(y_valid,valid_pred))}")

In [ ]:
#modeling(lr)

In [ ]:
#modeling(rf)

In [ ]:
#modeling(gb)

In [ ]:
from lightgbm import LGBMRegressor 
lgbm = LGBMRegressor(random_state=42)
modeling(lgbm)


In [ ]:
test_submission(X_test,lgbm)